In [ ]:
import numpy as np#matrix math
import cPickle as pickle# import load
import gym#allows you to train agents based on the game world




In [ ]:
# hyperparametrs
h = 200# no of hidden neurons
batch_size =  10# how many episodes we want in parametrer update
learning_rate = 1e-4# learning rate is very smalkl for convergence
gamma = 0.99 # discount factor opitimizing words for short term
decay_rate = 0.99# gradient descwent
# we re gonaa do this by hand
resume = False# do we wanna start from prevous model


In [ ]:
#init model
D = 80*80# input dimensionality # intialize aour model as 80*80 pixel value
# make it bigger to make pong bigger
if resume:# if we want to load from a previous data
    model = pickle.load(open('save.p', 'rb'))
else:
    model = {}
    # intialize their weights
    # smarter xavier intialization 
    # hidden node into account when intialization
    model['W1'] = np.random.randn(H, D)/np.sqrt(D)# smatrerv than random but actually random for tboth 
    # interval is from H to D divided by sqrt(D)
    # safely this formula
    # makes sure the weights are not too small or not too big to propagate the errors
    # like vainishing gradient problem
    # we do this for both set of weights
    model['W2'] = np.random.randn(H)/np.sqrt(H)
    # model is going to be a key value pair
    # key is going to be construct weights or layer the value should be weights inside it
    # the input has been converted into some vector
# intialise two more values
# gradient buffer and rms_prop_cash
# using buffers for gradients and rms_prop intializtaion for backprop storing intermediate values for gradients and computation for actual rms eqaution
grad_buffer = { k : np.zeros_like(v) for k,v in model.iteritems() }# for our gradient buffer we have a collection of zeroes and for each key value pair int he model # we should put the model in dictionary
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.iteritems()}# gradient buffer  is a way for to store gradients using backprop
#it is going to store the value of the formula in the rmsprop_cache 
# life is stochastic
# fate is determinstic static number
# variational autoencoders is not stochastic
# optimal batchsize to train ann - gridsearch

In [1]:
# activation function
def sigmoid(x):
    return 1.0/(1.0 + np.exp(-x))# squagshing

# then there is a preprocessing step
def prepro(I):
    # convert game train it into just what we need - the padding , the paddles , the ball
    I = I[35:195]# in general we see trial and error some subsample of image 
     # we need to crop it
    I = I[::2 , ::2  , 0]# DOWNFACTOR BY A FACTOR OF 2
    I = [ I==144 ] = 0# THIS IS THE TYPE OF THE BACKGROUND $ WE ERASE TO BACKGROUND
    I = [I==109]=0#THE OTHER BACKGROUND IS BEEN ERASED
    I[I !=0 ] =1 # PADDLES AND BALLS SET TO 1
    return I.astype(np.float).ravel()                    # we have to return the value as the flattened one dimensional array     

# we are optimizing for the short term and the way we do this by implementing this strategy called 
# we aregoing to recieve a set of rewards from a bunch of timestamps
# eevrytime our ball goes through one  direction , it goes past .... our game resets but the score doesnt .. we get rewards for everytime it is happening
# Weigh the each of the rewards differently
def discount_reward(r):   
    # An array of values that is going to feed that into the discount rewards
    # weigh of each of rewards differntly .. weigh the most immediate rewards higher than the next rewards
    discounted_r = np.zeros_like(r)# it is gonna start off like matrix of zeroes
    # discount reward formula
    # we have a discount rate and what we are doing weiging the rewards that are immediate , higher and next higher exponentially cause pong is a fast paced game
    # for every timestep the rewards are decresing in value of weight 
    # we are optimizing for the short term and we dont know what happen in long term
    # we will start off by intializing the running average or the reward sum and that will store the sum of the rewards 
    running_add = 0
    for t in reversed(range(0,r.size)):# for all values in the range of 0 to r.size
        if r[t] !=0:
            running_add =0
        # time to increment the sum
        # the sum over a set of values
        running_add = running_add + gamma + r[t]#discount reward factor - this helps us actually weigh those values in exponentially  decreasing order of time
        discounted_r = running_add 
    return discounted_r# short term means whenever ball is going past the imaging thats a short term reward continue hit the ball and there is a reward to all the actions 30 mins ago
    # lstm are great for text generation

def policy_forward(x):# foreard propagation chain of lineear algebra of matrix multipilcation all those output are fed to next layer get output probabilyy which is either up or down
    # then use that sample then use reward value to update our weights recursively that is why called policy gradients 
    # that we recieve from reward value
    # poilicy in the envoirment

    # get the input for pixels
    h = np.dot(model['W1'],x)
    # it gives a vector of values and squash into a linear of values
    h[h<0]=0 # relu # relu is used in mid of network while sigmoid is used for the end of network
    logp = np.dot(model['W2'], h)# dot product of next set of probablities and the hidden stste
    p = sigmoid(logp)
    # thrn we want to sample those probabilities and sample them to decide what we want to do
    # output probabilites for up down
    # sample from prob and decide what we want o do and get the policies by taking gradients wrt to the gradient values
    return p,h # return probability of taking action2 and hidden state
def policy_backward(eph,epdlogp):
    # recursively compute error derivatives for both of those layers
    # epdlogp - is going to modulate the event with advantage
    # we computing with weight 2 to start off with
    # eph is an array of hidden intermediate states
    # the ball passes the agent its considered a game and several games and episodes

    dW2 = np.dot(eph.T,epdlogp).ravel()
    dh = np.outer(epdlogp,model['W2'])
    dh[eph<=0]=0
    dW1 = np.dot(dh.T,epx)
    # return both derivatives to update weights
    return {'W1':dW1 , 'W2':dW2}
    # llongest commomn subsequence using ml .. sequence standard rnn not something like lsstm
    # deep learning on mulrtim deimensional
    # linera dimregreesion using sqaush dimensionality reduction technique  , like pca or tsme
    # takimg values atevery lasyer and using those value to compute values

# implemtnaion
env = gym.make('Pong-v0')
observation = env.reset()# we were going to take an obvservation from the enviorment 
# i like hardcoding things and when it comes to pixel values and game values i have never seen before , i have never worried about hardcoding things
# extract an pixel, observation from a gameset
prev_x = None
# we cant calculate motion if the frames are static , we an improve the motion of the ball
# we have to improve the motion of our ball if the frames are static
# we have to calculate the difference between the two frames and the difference is what we seen there
# we have to storet the previous value of diiff
# some intermediate variables to store the obseveation hidden sate gradient and reward
xs , hs, dlogps , drs = [], [],[], []
running_award = None
reward_sum = 0
episode_number = 0
# hyperparametrs dont affect the models architecture , they are used for computing the policy of the reward  not he actual reward itself


# begin traininng
# it training time
while True:
    # for training , we' re going to preprocess the information
    # get frame difference and for that we need the prev and curr frame
    cur_x = prepo(observation)# use the function that has been defined above
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x # now the prev_x is equal to curr_x
    #forwards
    aprob , h = policy_forward(x)# we need this policy forward method to sample from forward probability and the hidden state
    # sample from that probability then do back prop
    #stochastic
    action = 1 if np.random.uniform() < aprob else 3
    # sampling value is stochastic
    # stochasticity is in the model reparametriztrion technique
    dlog.append(y-aprob)# gradient that encorages that action that is taken to be taken

    # step the enviorment
    env.render()
    observation , reward , done , info = env.step(action)
    # standard the open ai format
    # we get the observationm , we get the reward , we get the info , binary value , the ball finished before
    reward_sum += reward# reward sum is going to store all the rewards

    drs.append(reward)# now we are gonna record the reward that has to be done after recall step after getting reward for the prevous action

    if done:# means the episode has finished then increment the episode number
        episode_number+=1
        

        # stack inputs hidden states actions rewards for the episodes compute the discounted reward back from
        epx = np.vstack(xs)
        eph = np.vstack(hs)
        edplogp = np.vstack(drs)
        # all those rewarsd into their respective stacks
        xs , hs , dlogps , drs = [],[],[],[]

        # discounted rewards computation
        discounted_epr = discount_reward(epr)# store the discounted reward in this variable epr
        # help control gradient estimator variance standarise the reward 
        discounted_epr -= np.mean(discounted_reward)# we take out the average value from these rewards
        # then we will average them all together 
        discounted_epr /= np.std(discounted_epr)

        # we are using this discounted reward to compute the gradient value
        epdlogp = discounted_epr
        # handcode pixels
        # pixel value for callls
        # we care about ball we care about palace
        
         # magnitiude has increased in 1 direction still we will be picking from either stoichasticlaly or determeinnstaclly still we 're going to pick one most likely
         # advantage says how good this action is compared to al;l the actions 
         # epdlogp is our gradient and disocunted epr is gonna act as our advantage
         #  the way is simple matrix multiplication 
         # this kinmd of reward of probality is backward denviorment 
         # final gradients to update weights by using rms weights
         # weird math operations 
         # the formula for rms prop 
         # rms prop is stochastic gradient descent
         # rms prop is the set of eqautons
         # we get gradient value and use the decay rate we backprop upadte ove rtime the values that we update closer to r=time are more than values update lesser in time
         # decy rate is related to siscount reawrd 
         # key steo update model key valuess
         # 
















    













     




    

 


SyntaxError: cannot assign to comparison (Temp/ipykernel_16360/4091712437.py, line 11)